IMPORT LIBRARIES

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

LOAD DATASETS

In [2]:
# Load the datasets
customers = pd.read_csv("/content/Customers.csv")
products = pd.read_csv("/content/Products.csv")
transactions = pd.read_csv("/content/Transactions.csv")

CONVERT TO DATETIME FORMAT

In [3]:
# Convert date columns to datetime format
customers['SignupDate'] = pd.to_datetime(customers['SignupDate'])
transactions['TransactionDate'] = pd.to_datetime(transactions['TransactionDate'])

In [4]:
# Aggregate transaction data to generate customer-level statistics
customer_stats = transactions.groupby('CustomerID').agg({
    'TotalValue': ['sum', 'mean'],
    'Quantity': 'sum',
    'TransactionDate': 'max'
}).reset_index()
customer_stats.columns = ['CustomerID', 'TotalExpenditure', 'AvgTransactionValue', 'TotalQuantity',
                          'LastTransactionDate']


In [18]:
customer_stats.head()

,CustomerID,TotalExpenditure,AvgTransactionValue,TotalQuantity,LastTransactionDate
0,C0001,3354.52,670.904,12,2024-11-02 17:04:16
1,C0002,1862.74,465.685,10,2024-12-03 01:41:41
2,C0003,2725.38,681.345,14,2024-08-24 18:54:04
3,C0004,5354.88,669.360,23,2024-12-23 14:13:52
4,C0005,2034.24,678.080,7,2024-11-04 00:30:22


In [5]:
# Merge customer stats with customer details
customers_merged = pd.merge(customers, customer_stats, on='CustomerID', how='left')

HANDLING MISSING VALUES

In [6]:
# Fill missing values with 0 for customers without transactions
customers_merged.fillna({
    'TotalExpenditure': 0,
    'AvgTransactionValue': 0,
    'TotalQuantity': 0,
    'LastTransactionDate': pd.Timestamp.min
}, inplace=True)


In [24]:
customers_merged.head()

,CustomerID,CustomerName,Region,SignupDate,TotalExpenditure,AvgTransactionValue,TotalQuantity,LastTransactionDate
0,C0001,Lawrence Carroll,South America,2022-07-10,3354.52,670.904,12.0,2024-11-02 17:04:16
1,C0002,Elizabeth Lutz,Asia,2022-02-13,1862.74,465.685,10.0,2024-12-03 01:41:41
2,C0003,Michael Rivera,South America,2024-03-07,2725.38,681.345,14.0,2024-08-24 18:54:04
3,C0004,Kathleen Rodriguez,South America,2022-10-09,5354.88,669.360,23.0,2024-12-23 14:13:52
4,C0005,Laura Weber,Asia,2022-08-15,2034.24,678.080,7.0,2024-11-04 00:30:22


In [7]:
# Extract product category preferences for each customer
customer_products = transactions.merge(products, on='ProductID')
product_preferences = customer_products.groupby(['CustomerID', 'Category'])['Quantity'].sum().unstack(fill_value=0)


In [20]:
customer_products.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,ProductName,Category,Price_y
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68


In [21]:
product_preferences.head()

Category,Books,Clothing,Electronics,Home Decor
CustomerID,,,,
C0001,2,0,7,3
C0002,0,4,0,6
C0003,0,4,4,6
C0004,8,0,6,9
C0005,0,0,4,3


In [8]:
# Combine customer data with product preferences
customer_data = pd.merge(customers_merged, product_preferences, on='CustomerID', how='left').fillna(0)


In [22]:
customer_data.head()

,CustomerID,CustomerName,Region,SignupDate,TotalExpenditure,AvgTransactionValue,TotalQuantity,LastTransactionDate,Books,Clothing,Electronics,Home Decor
0,C0001,Lawrence Carroll,South America,2022-07-10,3354.52,670.904,12.0,2024-11-02 17:04:16,2.0,0.0,7.0,3.0
1,C0002,Elizabeth Lutz,Asia,2022-02-13,1862.74,465.685,10.0,2024-12-03 01:41:41,0.0,4.0,0.0,6.0
2,C0003,Michael Rivera,South America,2024-03-07,2725.38,681.345,14.0,2024-08-24 18:54:04,0.0,4.0,4.0,6.0
3,C0004,Kathleen Rodriguez,South America,2022-10-09,5354.88,669.360,23.0,2024-12-23 14:13:52,8.0,0.0,6.0,9.0
4,C0005,Laura Weber,Asia,2022-08-15,2034.24,678.080,7.0,2024-11-04 00:30:22,0.0,0.0,4.0,3.0


FEATURE SELECTION

In [9]:
# Feature selection for similarity computation
features = ['TotalExpenditure', 'AvgTransactionValue', 'TotalQuantity'] + list(product_preferences.columns)


NORMALIZATION

In [10]:
# Normalize the features
scaler = StandardScaler()
customer_data_scaled = scaler.fit_transform(customer_data[features])

In [11]:
# Compute cosine similarity between customers
similarity_matrix = cosine_similarity(customer_data_scaled)

In [13]:
# Create a DataFrame for similarity scores
similarity_df = pd.DataFrame(similarity_matrix,
                             index=customer_data['CustomerID'],
                             columns=customer_data['CustomerID'])


In [14]:
# Function to get top 3 similar customers
def get_top_similar(customers_df, similarity_df, top_n=3):
    lookalike_map = {}
    for customer_id in customers_df['CustomerID'][:20]:  # First 20 customers (C0001 to C0020)
        similar_customers = similarity_df[customer_id].nlargest(top_n + 1).iloc[1:]  # Exclude self (highest similarity)
        lookalike_map[customer_id] = list(zip(similar_customers.index, similar_customers.values))
    return lookalike_map

In [15]:
# Get top 3 lookalikes for the first 20 customers
lookalike_map = get_top_similar(customers, similarity_df)


In [16]:
# Prepare the Lookalike.csv
lookalike_df = pd.DataFrame({
    'CustomerID': lookalike_map.keys(),
    'Lookalikes': [str(value) for value in lookalike_map.values()]
})


In [17]:
lookalike_df.to_csv("Kundanaa_Pilla_Lookalike.csv", index=False)

print("Lookalike Model and CSV generation complete.")

Lookalike Model and CSV generation complete.


In [23]:
lookalike_df.head()

,CustomerID,Lookalikes
0,C0001,"[('C0069', 0.919733109887925), ('C0120', 0.854..."
1,C0002,"[('C0031', 0.9202337133472378), ('C0189', 0.87..."
2,C0003,"[('C0195', 0.8434209191895081), ('C0166', 0.76..."
3,C0004,"[('C0065', 0.9461946558286309), ('C0075', 0.93..."
4,C0005,"[('C0095', 0.934289715897308), ('C0197', 0.930..."
